In [1]:
##########################################
# General helper functions and libraries #
##########################################

# General data libs
import pandas as pd
import numpy as np

# General plotting libs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation
import plotly.express as px
import seaborn as sns

####################
# Modeling process #
####################

# Networks
import networkx as nx
from pyvis.network import Network
from community import community_louvain

In [2]:
# Based on: https://www.bis.org/statistics/r_qa1406_hanx9b.pdf
# Originally from "Consolidated Banking Statistics, Updated 27 July 2023" described at: https://www.bis.org/statistics/consstats.htm
# Full set here: https://www.bis.org/statistics/full_data_sets.htm
# Retrieved on 13 Oct

# File, will take ~15sec to load
df = pd.DataFrame(pd.read_csv('https://storage.googleapis.com/suptech-lab-practical-data-science-public/WS_CBS_PUB_csv_col.csv'))
df.shape

(174575, 151)

In [3]:
df.columns

Index(['FREQ', 'Frequency', 'L_MEASURE', 'Measure', 'L_REP_CTY',
       'Reporting country', 'CBS_BANK_TYPE', 'CBS bank type', 'CBS_BASIS',
       'CBS reporting basis',
       ...
       '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4', '2022-Q1',
       '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1'],
      dtype='object', length=151)

In [4]:
# Immediate borrower basis
set(df['CBS reporting basis'])

{'Guarantor basis',
 'Immediate counterparty basis',
 'Net risk transfers (Inward-Outward)',
 'Outward risk transfers'}

In [5]:
# Amounts outstanding
set(df['Measure'])

{'Amounts outstanding / Stocks'}

In [6]:
set(df['Remaining maturity'])

{'Over 1 year and up to and including 2 years',
 'Over 2 years',
 'Total (all maturities)',
 'Up to and including 1 year'}

In [7]:
# List the columns we want to keep
cols = [
  'Reporting country',
  'Counterparty country',
  '2013-Q4',
  '2014-Q4',
  '2015-Q4',
  '2016-Q4',
  '2017-Q4'
]

# 1. Make a copy of the original dataframe
# 2. Filter only for the rows we want
# 3. Keep only the columns we want
df1 = df.copy(deep=True)
df1 = df1[(df1['CBS bank type']=="Domestic banks")
      & (df1['CBS reporting basis']=="Immediate counterparty basis")
      & (df1['Balance sheet position']=="Total claims")
      & (df1['Remaining maturity']=="Total (all maturities)")]
df1 = df1[cols]

# Drop the rows containing global aggregates
df1 = df1[(df1['Reporting country']!="All reporting countries") & (df1['Counterparty country']!="All reporting countries") &
         (df1['Counterparty country']!='All countries (total)') & (df1['Counterparty country']!='All countries excluding residents') &
         (df1['Counterparty country']!='International organisations') & (df1['Counterparty country']!='International organisations') &
         (df1['Counterparty country']!='Residents/Local') & (df1['Counterparty country']!='Residents/Local') &
         (df1['Counterparty country']!='Euro area') & (df1['Counterparty country']!='Euro area')]

# Only those who have recorded financial data for the quarter we're looking at
df1 = df1[pd.notnull(df1['2013-Q4'])]

df1


,Reporting country,Counterparty country,2013-Q4,2014-Q4,2015-Q4,2016-Q4,2017-Q4
113,Australia,Chad,0.008,0.741,0.301,NaN,NaN
172,Switzerland,Chad,2.028,18.635,0.212,0.021,0.013
234,France,Chad,383.000,662.000,525.000,465.000,491.000
424,United States,Chad,35.000,46.000,44.000,38.000,39.000
513,Austria,Germany,39068.469,37377.994,37690.878,35869.525,42703.591
...,...,...,...,...,...,...,...
174270,Denmark,Djibouti,1.000,1.000,1.000,1.000,1.000
174303,France,Djibouti,30.000,58.000,10.000,14.000,12.000
174335,United Kingdom,Djibouti,21.000,17.000,12.000,6.000,NaN
174490,Sweden,Djibouti,18.168,15.395,7.716,1.542,0.351


In [8]:
set(df1['Reporting country'])

{'Australia',
 'Austria',
 'Belgium',
 'Brazil',
 'Canada',
 'Chile',
 'Chinese Taipei',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Ireland',
 'Italy',
 'Japan',
 'Korea',
 'Mexico',
 'Netherlands',
 'Panama',
 'Portugal',
 'Spain',
 'Sweden',
 'Switzerland',
 'Türkiye',
 'United Kingdom',
 'United States'}

In [9]:
# Add nodes
G = nx.from_pandas_edgelist(df1, 'Reporting country', 'Counterparty country', edge_attr=True, create_using=nx.DiGraph)
print("Nodes:", len(G.nodes()), "Edges:", len(G.edges()))

Nodes: 215 Edges: 2674
